[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Humboldt-WI/IPML/blob/master/tutorial_notebooks/t7_classification.ipynb)

# Classification Models: The case of credit default prediction
<hr>

In [11]:
# Standard imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

## Data preparation
In this tutorial, we revisit the logistic regression model and study how it allows us to approach binary classification problems. To that end, we consider the case of credit default prediction considering the *Home Equity (HMEQ)* data set from the famous [Credit Risk Analytics](http://www.creditriskanalytics.net) textbook. It comprises information about a set of borrowers, which are categorized along demographic features and features concerning their business relationship with the lender. A binary target variable called 'BAD' is provided and indicates whether a borrower has repaid their debt. Here is an overview of the data:
- BAD: the target variable, 1=default; 0=non-default
- LOAN: amount of the loan request
- MORTDUE: amount due on an existing mortgage
- VALUE: value of current property
- REASON: DebtCon=debt consolidation; HomeImp=home improvement
- JOB: occupational categories
- YOJ: years at present job
- DEROG: number of major derogatory reports
- DELINQ: number of delinquent credit lines
- CLAGE: age of oldest credit line in months
- NINQ: number of recent credit inquiries
- CLNO: number of credit lines
- DEBTINC: debt-to-income ratio

As you can see, the features aim at describing the financial situation of a borrower, which should probably tell us something about the risk of a borrower to default.

Using the `Pandas`library, we can retrieve the data right from the web; specifically the GitHub repository of our course. 

In [12]:
url = 'https://raw.githubusercontent.com/Humboldt-WI/bads/master/data/hmeq.csv'
df = pd.read_csv(url)  # standard pandas function to load tabular data in CSV format
df.head(5)  # obtain a preview of the data by showing the first 5 rows of the table

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,1,1100,25860.0,39025.0,HomeImp,Other,10.5,0.0,0.0,94.366667,1.0,9.0,NaN
1,1,1300,70053.0,68400.0,HomeImp,Other,7.0,0.0,2.0,121.833333,0.0,14.0,NaN
2,1,1500,13500.0,16700.0,HomeImp,Other,4.0,0.0,0.0,149.466667,1.0,10.0,NaN
3,1,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,1700,97800.0,112000.0,HomeImp,Office,3.0,0.0,0.0,93.333333,0.0,14.0,NaN


### Missing Values
As shown above, the data exhibits missing values. We have discussed missing values in the scope of our [EDA lecture](https://moodle.hu-berlin.de/pluginfile.php/5910736/mod_resource/content/2/ipml_s3_eda.pdf). This is not the time to revisit that material. To use the data set, we simply replace missing values using the median and mode of numerical and categorical features respectively.   

In [13]:
# Missing value handling
print('When starting, the data exhibits {} missing values'.format(df.isna().sum().sum()))

for col in df.columns:  # loop through all the columns (i.e., features)
    if df[col].dtype == 'O':  # decide on the imputation strategy based on the data type
        df[col].fillna(df[col].mode()[0], inplace=True)  # model replacement for categories
    else:
        df[col].fillna(df[col].median(), inplace=True)  # Mean replacement for all other features 

# Verify that the data frame does not exhibit missing values anymore
print('At this point, the data exhibits {} missing values'.format(df.isna().sum().sum()))

When starting, the data exhibits 5271 missing values
At this point, the data exhibits 0 missing values


In [14]:
# Preview the data
df.head(5)

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,1,1100,25860.0,39025.0,HomeImp,Other,10.5,0.0,0.0,94.366667,1.0,9.0,34.818262
1,1,1300,70053.0,68400.0,HomeImp,Other,7.0,0.0,2.0,121.833333,0.0,14.0,34.818262
2,1,1500,13500.0,16700.0,HomeImp,Other,4.0,0.0,0.0,149.466667,1.0,10.0,34.818262
3,1,1500,65019.0,89235.5,DebtCon,Other,7.0,0.0,0.0,173.466667,1.0,20.0,34.818262
4,0,1700,97800.0,112000.0,HomeImp,Office,3.0,0.0,0.0,93.333333,0.0,14.0,34.818262


### Category encoding
The data set comprises two categorical variables. We must encode this before moving on. *Dummy coding* is a standard approach to do so. Dummy coding is best explained by examining how it chances our two categorical variables REASON and JOB.

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5960 entries, 0 to 5959
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   BAD      5960 non-null   int64  
 1   LOAN     5960 non-null   int64  
 2   MORTDUE  5960 non-null   float64
 3   VALUE    5960 non-null   float64
 4   REASON   5960 non-null   object 
 5   JOB      5960 non-null   object 
 6   YOJ      5960 non-null   float64
 7   DEROG    5960 non-null   float64
 8   DELINQ   5960 non-null   float64
 9   CLAGE    5960 non-null   float64
 10  NINQ     5960 non-null   float64
 11  CLNO     5960 non-null   float64
 12  DEBTINC  5960 non-null   float64
dtypes: float64(9), int64(2), object(2)
memory usage: 605.4+ KB


In [16]:
df = pd.get_dummies(data=df, drop_first=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5960 entries, 0 to 5959
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   BAD             5960 non-null   int64  
 1   LOAN            5960 non-null   int64  
 2   MORTDUE         5960 non-null   float64
 3   VALUE           5960 non-null   float64
 4   YOJ             5960 non-null   float64
 5   DEROG           5960 non-null   float64
 6   DELINQ          5960 non-null   float64
 7   CLAGE           5960 non-null   float64
 8   NINQ            5960 non-null   float64
 9   CLNO            5960 non-null   float64
 10  DEBTINC         5960 non-null   float64
 11  REASON_HomeImp  5960 non-null   bool   
 12  JOB_Office      5960 non-null   bool   
 13  JOB_Other       5960 non-null   bool   
 14  JOB_ProfExe     5960 non-null   bool   
 15  JOB_Sales       5960 non-null   bool   
 16  JOB_Self        5960 non-null   bool   
dtypes: bool(6), float64(9), int64(2)


In [17]:
# Create our standard variables X,y for subsequent tasks
X = df.copy()
y = X.pop('BAD')

## Exercise 1: Plotting data for classification
You will remember the many plots we came across when discussing regression. We also saw some analog plots for classification problems in the [lecture](https://moodle.hu-berlin.de/pluginfile.php/5941409/mod_resource/content/1/ipml_s6_classification.pdf). One of them was a 2d scatter plot displaying the bi-variate relationship between one feature and the binary target variable. 
<br>
<img alt="Classification problem in 2D" src="https://raw.githubusercontent.com/Humboldt-WI/demopy/main/1d_classification_problem.PNG" width=600 height=800/>
<br><br>
Another was the 2d scatter plot showing two selected features and distinguishing examples of the two classes using colors.
<br>
<img alt="Classification problem in 2D" src="https://raw.githubusercontent.com/Humboldt-WI/demopy/main/2d_classification_problem.png" width=600 height=800/>
<br><br>

Your first task is to create a similar plots for the credit data. In principle, you can select any combination of features that you like.  

In [ ]:
# Exercise 1:


## Exercise 2: Model adequacy  
We introduced logistic regression as an extension of linear regression for cases in which we work with a binary target variable. Nonetheless, just as linear regression, logistic regression is a (generalized) linear model. It assumes that feature values determine the target via a linear, additive functional relationship.

Play with the above code to plot the distribution of different combinations of the features. Eventually, you should arrive at a preliminary conclusion of whether logistic regression is a suitable model for the credit risk data at hand. 

## Exercise 3: Logistic regression
Time to estimate our first model. Unsurprisingly, our goto library `sklearn` supports logistic regression through the class `LogisticRegression`, which is available in the module `sklearn.linear_models`. 

A nice feature of `sklearn`is that it facilitates using different ML algorithms through a common interface. This means that the way in which you use, for example train and test, a model is basically the same no matter which learning algorithm you use. Last week, we examined the implementation of linear regression. So, the way to use logistic regression should be very similar. Feel free to draw on the examples from last week and/or the corresponding [solution notebook](https://github.com/Humboldt-WI/IPML/blob/main/tutorial_notebooks/t6_regression_solution.ipynb). You task is to:
- partition our data into a 75% training and 25% test set,
- train a logistic regression model on the training set,
- and compute predictions for the test set. 

### Model training

Note that the `sklearn` implementation does not provide an informative summary, as did the library `statsmodels`, which you saw in our [regression notebook](https://github.com/Humboldt-WI/IPML/blob/main/tutorial_notebooks/t6_regression_solution.ipynb). Basically, the reason is that `sklearn` is designed to support prediction. Let's demonstrate how to do this, that is compute predictions using the trained model.

### Test set predictions

## Exercise 4: Diagnosing predictions

### 4a) Classification accuracy and the score function
Likely, you are also interested in assessing the model. There is an easy way to do this. Just call the function `score()`, which the trained logistic regression model supports. More specifically, assume the variable that you created above is called `lr_model`, and assume further that your feature matrix and target variable are called `X` and `y`, respectively. Then, you can run:
```
lr_model.score(X,y)
```
Try it out but remember to adjust the names of the variables appropriately. 

In [ ]:
# Compute the score of the logistic regression model


Supposedly, you will have observed a score of about 80 percent, which does sound pretty good, right? Well, to be sure we should better understand for start what this number means, that is, what *score* we have just calculated. It turns out that when you call the `score()` function for a classification model, you receive an estimate of that model's classification accuracy as result. Regression models would produce a different *score*. To verify your score is indeed the classification accuracy, recalculate classification *manually*. In case, recall how we defined classification accuracy in the lecture:

<br>
<img alt="Classification problem in 2D" src="https://raw.githubusercontent.com/Humboldt-WI/demopy/main/confusion_matrix.PNG" width=600 height=800/>
<br><br>

In [ ]:
# Code to compute (test set) accuracy


### 4b) Comparing to a naive classifier
Interpreting classification accuracy, and more generally the performance of any predictive model, it is useful to compare is to a baseline. But what baseline? We face a classification problem. There are two classes, good payers and bad payers, and we aim to tell these apart. Come up with a very basic - naive - strategy to solve the classification problem without using any model. Write a piece of code to calculate the classification accuracy of your naive strategy and compare to logistic regression. 
> Hint: if you feel a bit lost, consider web searching for *dummy classifier* 

In [ ]:
# Code to calculate the score of a naive classifier or dummy classifier


### 4c) Confusion matrix
The comparison of logistic regression to a naive classifier should have revealed that your model is not that strong after all. Classification accuracy of almost 80% sounds good at first glance but is far less impressive when knowing that naively predicting the majority class gives basically the same result. To get a better understanding of how well (or poorly) the model classifiers, your next task is to produce a confusion matrix. You can find all the functionality that you need in the class `ConfusionMatrixDisplay`, which is part of the `sklearn.metrics` module. So, import that class, examine how it works using the documentation, and plot a beautiful confusion matrix for your logistic regression model. Once you acomplished this task, make sure to study the result and discuss the quality of the classifier.

## Exercise 5: ROC Analysis
Previous analysis suggests that the logistic regression model was not doing a good job. It barely recognizes the bad risks, which is the key objective of a credit default prediction model. That said, don't be too quick with dismissing the model. Recall that the confusion matrix is based on a cut-off value:

<br>
<img alt="Classification problem in 2D" src="https://raw.githubusercontent.com/Humboldt-WI/demopy/main/classification_cutoff.PNG" width=600 height=800/>
<br><br>

Up to this point, we did not set the cut-off value anywhere. Given this, it is actually surprising that we have been able to create a confusion matrix in the previous exercise. The reason for this is that `sklearn` assumes a default cut-off value of 0.5. This cut-off is used whenever you call the function `predict()` for a classifier. In our opinion, this makes the `predict()` function rather useless for classification. For classification problems, you should routinely compute probability predictions. If you seek discrete class predictions, compare the model-based probability predictions to a cut-off; as illustrated in the above picture. How to choose a **suitable** cut-off is a nontrivial question and out-of-the-scope of this notebook. Rather, as the title of the exercise has already indicated, we want to revisit ROC analysis. To that end, produce probability forecasts for the test data set using your logistic regression model.

### 5a) Probability predictions
Compute probabilistic predictions using the function `predict_proba()`. Create a variable `yhat_prob` to store the results.

Note how predict_proba returns the probabilities of both classes. This appears awkward. Given they add up to one, why have both? The answer is that `sklearn` aims to be consistent. Whenever you face a multi-class problem, it is essential to return the estimated probabilities of each class. Thus, for the sake of consistency, the implementation of the `predict_proba()` function is such that it returns probability estimates for each class.

### 5b) ROC curve
The probabilistic predictions are the foundation of the ROC curve. Roughly speaking, we compare them to all possible cut-off values, derive all possible confusion matrices, extract the true positive rate and false positive rates from these confusion matrixes, and plot those values against one another. Of course, we do not have to do this by hand. Instead, sklearn provides the corresponding functionality via several functions and classes including `roc_curve`, `RocCurveDisplay()`, and, if you only care about the AUC score, `roc_auc_score`. All of them are available in the module `sklearn.metrics`. Your task is to import these functions and familiarize yourself with the functionality. Begin with the class `RocCurveDisplay()`, which should quickly produce you a nice ROC curve. Afterward, try to augment your curve using the diagonal line. A discussed in the lecture, a diagonal line in ROC space represents the performance of guessing classes at random. Next, we create the plot using the function `roc_curve`, which offers more flexibility but also requires you to write more code. 

### 5c) K-fold cross-validation
Recall that $K$-fold Cross-Validation allows us to evaluate a model's performance and generalization capability. The process involves dividing the dataset into $K$ subsets or "folds". The model is then trained on $K-1$ folds and tested on the remaining one, repeating this process $K$ times. The resulting $K$ performance scores provide a robust assessment of the model's overall capability, making it an essential practice when developing machine learning models using Python and similar languages. 

<img alt="Cross-validation" src="https://raw.githubusercontent.com/Humboldt-WI/demopy/main/cross_validation1.png" width=600 height=800/>

Your task is to apply $K$-fold cross-validation using the `KFold()` class from the module `sklearn.model_selection` to get a robust estimate of the AUC ROC. First import the `KFold()` class. Using its documentation, web search, and perhaps GenAI, write code to cross-validate the logistic regression.

In [ ]:
# Cross-validation of the logistic regression model


## The visual logistic regression
We complete this part with a visual demo of the logistic regression model. No more work for you, just execute the code and enjoy. And of discuss the plot to make sure you understand how it depicts the classification solution of our logistic regression model. 

In [ ]:
def plot_logit_decision_surface(model, y, data, x1, x2, save_fig=False):
    '''
        Visualization of logistic regression in 2D
        
        Creates a plot depicting the distribution of the input
        data along two dimensions and the probability predictions
        of a logistic regression model. 

        Parameters
        ----------
        model :   An instance of the sklearn class LogisticRegression,  which        
                  has been trained on the input data.

        y     :   The true outcomes of the target variable 
        
        data  :   Pandas data frame providing the feature values.

        x1, x2:   The function plots the results of logistic regression in
                  two dimensions. The parameters x1 and x2 give the names
                  of the features used for plotting. These features will be
                  extracted from the data frame.

        save_fig: Binary variable allowing you to save the figure as a PNG image. 
                  Default: False

        Returns
        ----------
        The function does not return a result. It's purpose is to visualize 
        logistic regression model. The corresponding plot is the only output.
    '''

    #if len(model.coef_.ravel())!=2:
    #    raise Exception('Please estimate a logit model using only two features!')
    # Define some variables to govern the plot
    bounds = data.describe().loc[["min", "max"]][[x1, x2]].to_numpy()  # value ranges of the two features
    eps = 5  # tolerance parameter 

    # Create hypothetical data points spanning the entire range of feature values.
    # We need these to get from our logistic regression model a probability prediction
    # for every possible data point
    xx, yy = np.mgrid[(bounds[0,0]-eps):(bounds[1,0]+eps), (bounds[0,1]-eps):(bounds[1,1]+eps)]
    grid = np.c_[xx.ravel(), yy.ravel()]

    # Perhaps the logistic regression model was fitted using the full data frame. 
    # To also work in that case, we extract the estimated regression coefficients 
    # corresponding to the two features we consider for plotting
    feature_to_index = {name: idx for idx, name in enumerate(model.feature_names_in_)}  # create a dic as intermediate step
    indices = [feature_to_index[f] for f in [x1, x2]]  # Find the indices of our two features of interest using the dic
    w = model.coef_.ravel()[indices]  # estimated regression coefficients
    b = model.intercept_  # estimated intercept of the logistic regression model

    # Compute probability predictions over the entire space of possible feature values
    # In the interest of robustness, we manually compute the logistic regression predictions
    # using the regression coefficients extracted above
    probs = 1/(1+np.exp(-(np.dot(grid, w.reshape(2,-1))+b))).reshape(xx.shape)

    # We are finally ready to create our visualization
    f, ax = plt.subplots(figsize=(8, 6))  # new figure
    # Contour plot of the probability predictions across the entire feature range
    contour = ax.contourf(xx, yy, probs, 25, cmap="RdBu", vmin=0, vmax=1)  
    ax_c = f.colorbar(contour)
    ax_c.set_label("$\hat{p}(y=1|X)$")
    ax_c.set_ticks([0, .25, .5, .75, 1])

    # Scatter plot of the actual data
    ax.scatter(data[x1], data[x2], c=y, s=50, cmap="RdBu", vmin=0, vmax=1,
               edgecolor="white", linewidth=1);
    plt.xlabel(x1)
    plt.ylabel(x2)
    if save_fig==True:
        plt.savefig('logit_contour.png')
    plt.show()


# Calling the function
plot_logit_decision_surface(lr_model, ytr, Xtr, 'DEBTINC', 'YOJ')

# Well done! This was another comprehensive set of exercises.